In [21]:
import numpy as np
import pandas as pd 
import cv2 
import os 

# current working directory
print(os.getcwd()) # /sim_ws/src/global_planner/maps

# set map name
map_name = 'HRL'

# import centerline csv
centerline_path = os.path.join(os.getcwd(), map_name, f'{map_name}_centerline.csv')
centerline_csv = pd.read_csv(centerline_path)

# convert to numpy array for better / easier calculation 
centerline = centerline_csv[['x', 'y']].to_numpy()
print(centerline)

/sim_ws/src/global_planner/maps
[[1449.  199.]
 [1448.  200.]
 [1447.  200.]
 ...
 [1452.  199.]
 [1451.  199.]
 [1450.  199.]]


In [22]:
# use only every 20th centerline point
step_size = 20
centerline_2 = centerline[::step_size]
print('before:', centerline.shape[0], 'after:', centerline_2.shape[0])

before: 3157 after: 158


In [23]:

import math 

threshold = 0.04

# calculate the theta between centerline points
def compute_centerline_with_theta(centerline_xy: np.ndarray):
     
    centerline_with_theta = np.zeros(shape=(centerline_xy.shape[0], 3))

    for i in range(centerline_with_theta.shape[0]):
        if i == centerline_with_theta.shape[0]-1:
            x2 = centerline_xy[0][0]
            y2 = centerline_xy[0][1]
        else:
            x2 = centerline_xy[i+1][0]
            y2 = centerline_xy[i+1][1]
        x1 = centerline_xy[i][0]
        y1 = centerline_xy[i][1]
        
        theta = math.atan2(y2-y1, x2-x1)
        centerline_with_theta[i] = np.array([x1,y1,theta])
    return centerline_with_theta

# keep only relevant centerline points 
def filter_centerline_points(centerline: np.ndarray):
    indices_to_remove = []
    for i in range(centerline.shape[0]):
        if i == centerline.shape[0]-1:
            theta2 = centerline[0][2]
            theta0 = centerline[i-1][2]
        elif i == 0:
            theta2 = centerline[i+1][2]
            theta0 = centerline[centerline.shape[0]-1][2]
        else:
            theta2 = centerline[i+1][2]
            theta0 = centerline[i-1][2]
        theta1 = centerline[i][2]
        theta01_error = theta1-theta0
        theta01_error = (theta01_error + math.pi) % (2 * math.pi)  - math.pi
        theta12_error = theta2-theta1
        theta12_error = (theta12_error + math.pi) % (2 * math.pi)  - math.pi

        if abs(theta01_error) < threshold and abs(theta12_error < threshold):
            indices_to_remove.append(i)
    
    filtered_centerline = np.delete(centerline, indices_to_remove, axis=0)

    return filtered_centerline
            

# while loop until it converges (so no points are removed in an iteration)  
centerline_3 = centerline_2.copy()

while True:
    centerline_4 = compute_centerline_with_theta(centerline_3)
    centerline_4 = filter_centerline_points(centerline_4)
    print(centerline_3.shape[0], centerline_4.shape[0])
    if centerline_3.shape[0] == centerline_4.shape[0]:
        print('convergation')
        break
    else:
        centerline_3 = centerline_4.copy()
        

158 134
134 134
convergation


In [24]:
# starting pose for map yaml 
print('starting pose:', centerline_4[0])
# save points in csv file as filtered centerline 
centerline_final = centerline_4[:, :2] # remove theta column
centerline_filtered = pd.DataFrame(centerline_final, columns=['x','y'])
centerline_filtered_path = os.path.join(os.getcwd(), map_name, f'{map_name}_centerline_eff.csv')
centerline_filtered.to_csv(centerline_filtered_path, index=False)

starting pose: [1449.          199.            2.85013586]
